## Trabalho final desenvolvido pelos alunos:
- Itor Carlos Souza Queiroz
- Lanna Luara Novaes Silva

### Instalação das dependências necessárias

In [ ]:
%pip install -r requirements.txt  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import io, color, filters, feature
from skimage.util import img_as_float
from skimage.filters import sobel
from skimage.io import imread
import zipfile
import os
import requests
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.filters import sobel, threshold_otsu



### Define o diretório onde ficará as imagens do dataset que está sendo usado no trabalho

In [ ]:
caminho_diretorio = 'dices/'
arquivos = [f for f in os.listdir(caminho_diretorio) if f.endswith(('.png', '.jpg', '.jpeg'))]

### Cria uma função para ler as imagens da pasta "dices" para exibir as imagens antes de realizar as próximas operações

In [ ]:
def exibe_grade(lista_imagens, titulos, mapa_cor=None):
    n_imagens = len(lista_imagens)
    numero_colunas = 4
    numero_linhas = (n_imagens // numero_colunas) + (1 if n_imagens % numero_colunas != 0 else 0)

    plt.figure(figsize=(15, numero_linhas * 4))
    for i, imagem in enumerate(lista_imagens):
        plt.subplot(numero_linhas, numero_colunas, i + 1)
        plt.imshow(imagem, cmap=mapa_cor if len(imagem.shape) == 2 else None)
        plt.axis("off")
        plt.title(titulos[i])
    plt.tight_layout()
    plt.show()

### Definição de uma função para converter as imagens para a escala de cinza. Primeiramente será removido o canal alpha, referente a transparência, caso o mesmo exista.

In [ ]:
def converter_para_cinza(lista_entrada):
    lista_saida = []
    for img in lista_entrada:
        # Remove canal alpha (transparência) se existir
        if len(img.shape) == 3 and img.shape[-1] == 4:
            img = img[:, :, :3]

        cinza = color.rgb2gray(img)
        lista_saida.append(cinza)
    return lista_saida

### Exibição das imagens originais, ou seja, anterior as operações do trabalho

In [ ]:
imagens_originais = []
for nome in arquivos:
    img = io.imread(os.path.join(caminho_diretorio, nome))
    imagens_originais.append(img)


print("Exibindo imagens originais...")
exibe_grade(imagens_originais, arquivos)

### Aplicação da operação de conversão para a escala de cinza e a exibição das imagens do mesmo

In [ ]:
imagens_cinza = converter_para_cinza(imagens_originais)

print("Exibindo imagens em escala de cinza...")
# Importante: para imagens em cinza, passamos cmap='gray'
exibe_grade(imagens_cinza, arquivos, mapa_cor='gray')

### Aplicação da operação de suavização em cada imagem do dataset e a exibição das mesmas após a operação.

In [ ]:
# Aplicando a suavização com sigma=1.5 em todas as imagens
imagens_suavizadas = [filters.gaussian(img, sigma=1.5) for img in imagens_cinza]

print(f"{len(imagens_suavizadas)} imagens foram suavizadas com sucesso!")

# Exibindo o resultado usando a função que criamos anteriormente
print("Exibindo imagens suavizadas (Noise Reduction):")
exibe_grade(imagens_suavizadas, arquivos, mapa_cor='gray')

### Definição de operações para realizar a extração e binarização da borda.


In [ ]:
def extrair_bordas_sobel(imagem):
    """Aplica o filtro Sobel para realçar transições de intensidade."""
    return sobel(imagem)

def binarizar_bordas(imagem_sobel):
    """Aplica o limiar de Otsu para converter a imagem em preto e branco (bordas)."""
    thresh = threshold_otsu(imagem_sobel)
    return imagem_sobel > thresh

In [ ]:
def encontrar_circulos_hough(imagem_binaria, raio_min, raio_max, passo=1, total_picos=5):
    """Executa a Transformada de Hough para detectar círculos em um intervalo de raios."""
    hough_radii = np.arange(raio_min, raio_max, passo)
    hough_res = hough_circle(imagem_binaria, hough_radii)

    # Seleciona os picos mais evidentes
    accums, cx, cy, radii = hough_circle_peaks(
        hough_res, hough_radii, total_num_peaks=total_picos
    )
    return cx, cy, radii

def processar_imagem_dado(img, raio_min, raio_max):
    """Pipeline principal para processar uma única imagem."""
    # 1. Realce de bordas
    img_sobel = extrair_bordas_sobel(img)

    # 2. Binarização
    img_binaria = binarizar_bordas(img_sobel)

    # 3. Detecção
    cx, cy, radii = encontrar_circulos_hough(img_binaria, raio_min, raio_max)

    return img_sobel, (img, cx, cy, radii)

def detectar_circulos_batch(imagens_suavizadas, raio_min, raio_max):
    """Função orquestradora para processar a lista de imagens."""
    imagens_sobel = []
    resultados_visualizacao = []

    for img in imagens_suavizadas:
        img_sobel, res_viz = processar_imagem_dado(img, raio_min, raio_max)

        imagens_sobel.append(img_sobel)
        resultados_visualizacao.append(res_viz)

    return imagens_sobel, resultados_visualizacao

### Operação de detectação de bordas dos objetos de cada imagem presente no dataset escolhido.

In [ ]:
imagens_sobel = []
imagens_binarias  = []

def detectar_circulos(imagens_suavizadas, raio_min, raio_max, passo=1):
    resultados_visualizacao = []

    # Definimos um intervalo de raios para procurar
    hough_radii = np.arange(raio_min, raio_max, passo)

    for img in imagens_suavizadas:
        # 1. Aplicando Sobel para realçar bordas
        # O Sobel ajuda a destacar a transição entre o dado e os pontos
        aplicacao_sobel = sobel(img)
        imagens_sobel.append(aplicacao_sobel)

        # 2. Binarização manual com limiar fixo
        limiar_fixo = 0.05  # Limiar definido manualmente
        bordas_binarias = aplicacao_sobel > limiar_fixo

        
        imagens_binarias.append(bordas_binarias)

        # 3. Transformada de Hough Circular
        hough_res = hough_circle(bordas_binarias, hough_radii)

        # Selecionar os picos mais evidentes (os 5 círculos mais prováveis)
        accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=5)

        # Guardamos a imagem original e os dados dos círculos para plotar
        resultados_visualizacao.append((img, cx, cy, radii))

    return resultados_visualizacao

# Ajuste os raios de acordo com o tamanho dos objetos nas suas imagens de 'dices'
resultados = detectar_circulos(imagens_suavizadas, raio_min=6, raio_max=12)

In [ ]:
print("Exibindo imagens com filtro de Sobel aplicado:")
exibe_grade(imagens_sobel, arquivos, mapa_cor='gray')

### Aplicação do filtro de Sobel e exibição das imagens com bordas realçadas.

In [ ]:
print("Exibindo imagens após a aplicação de binarização:")
exibe_grade(imagens_binarias, arquivos, mapa_cor='gray')

### Exibição das imagens após a aplicação de Hough

In [ ]:
def exibe_resultados_hough(resultados):
    n_imagens = len(resultados)
    fig, axes = plt.subplots(nrows=(n_imagens // 4) + 1, ncols=4, figsize=(15, 10))
    axes = axes.ravel()

    for i, (img, cx, cy, radii) in enumerate(resultados):
        axes[i].imshow(img, cmap='gray')
        for center_y, center_x, radius in zip(cy, cx, radii):
            circ = plt.Circle((center_x, center_y), radius, color='yellow', fill=False, lw=2)
            axes[i].add_patch(circ)
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

exibe_resultados_hough(resultados)